In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
%%time
from nngp import NN
from dataloader.rosen import RosenData 
import numpy as np
import tensorflow as tf

CPU times: user 12 µs, sys: 2 µs, total: 14 µs
Wall time: 15.3 µs


In [26]:
config = {
    'random_state': 4623457,
    'ndim': 10,
    'N_train': 200,
    'N_test': 200,
    'N_pool': 1000,
    'layers': [128, 64, 32],
    'update_sample_size': 10
}



In [12]:
np.random.seed(config['random_state'])

In [13]:
X_train, y_train, _, _, X_test, y_test, X_pool, y_pool = RosenData(
    config['N_train'], 0, config['N_test'], config['N_pool'], config['ndim']
).dataset()

print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

shapes: (200, 10) (200, 1)
shapes: (200, 10) (200, 1)
shapes: (1000, 10) (1000, 1)


In [19]:
# tf.reset_default_graph()
# nn = NN(
#     ndim = config['ndim'],
#     random_state = config['random_state'],
#     layers = config['layers']
# )
tf.reset_default_graph()
from model.mlp import MLP

nn = MLP(
    ndim = config['ndim'],
    random_state = config['random_state'],
    layers = config['layers']
)

In [20]:
try:
    sess.close()
except:
    pass
# a setting for my cluster; ignore it
session_config = tf.ConfigProto()
session_config.gpu_options.allow_growth = True

# global init
init = tf.global_variables_initializer()
# saver = tf.train.Saver()
sess = tf.Session(config=session_config)
sess.run(init)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [21]:
nn.train(
    session = sess,
    X_train = X_train,
    y_train = y_train,
    X_test = X_test,
    y_test = y_test,
    X_val = X_test,
    y_val = y_test,
)

[100] RMSE train:186.437 test:191.325 val:191.325 patience:3
[200] RMSE train:181.961 test:186.756 val:186.756 patience:3
[300] RMSE train:169.345 test:173.799 val:173.799 patience:3
[400] RMSE train:142.603 test:146.337 val:146.337 patience:3
[500] RMSE train:103.516 test:106.335 val:106.335 patience:3
[600] RMSE train:72.867 test:75.470 val:75.470 patience:3
[700] RMSE train:65.293 test:68.313 val:68.313 patience:3
[800] RMSE train:64.714 test:67.930 val:67.930 patience:3
[900] RMSE train:64.535 test:67.832 val:67.832 patience:3
[1000] RMSE train:64.356 test:67.718 val:67.718 patience:3
[1100] RMSE train:64.169 test:67.597 val:67.597 patience:3
[1200] RMSE train:63.975 test:67.472 val:67.472 patience:3
[1300] RMSE train:63.775 test:67.344 val:67.344 patience:3
[1400] RMSE train:63.570 test:67.213 val:67.213 patience:3
[1500] RMSE train:63.361 test:67.079 val:67.079 patience:3
[1600] RMSE train:63.148 test:66.945 val:66.945 patience:3
[1700] RMSE train:62.932 test:66.809 val:66.809 pa

(10000, 9.948867, 30.142544, 30.142544)

In [25]:
from uncertainty_estimator.mcdue import MCDUE

estimator = MCDUE(nn)

uncertainties = estimator.estimate(sess, X_pool)
uncertainties

array([146.4973672 , 160.96291032, 123.44207888, 200.93277094,
       197.45647162, 133.90289454, 106.04157324, 133.33457382,
       144.59799363, 155.11296718, 129.15270999, 131.6564702 ,
       122.38041243, 228.73247927, 180.4970015 , 194.06952367,
       148.94616696, 158.31557482, 131.8225332 , 126.40987907,
       171.9334208 , 139.20758116, 108.56011468, 105.29852235,
       152.68761414, 125.65620625, 175.39934572, 152.45251574,
       133.19957863, 138.90442602, 100.81538717, 107.62164213,
       149.08543256, 122.20930045, 131.48093505, 155.92415817,
        84.30262699, 143.14078618, 206.82888404, 120.28345986,
       217.3891008 , 193.43712376, 156.06986782, 102.42699606,
       136.84683637, 134.43706997, 131.62919943, 156.52908157,
       110.87883598, 113.92748633, 159.88802627, 220.13707598,
       123.3013962 , 111.69145434, 166.58435137, 157.14663054,
        98.57476176, 162.19602437, 206.28320265, 127.77208196,
       132.74387738, 110.8496644 , 159.61100555, 182.83

In [28]:
from sample_selector.eager import EagerSampleSelector

selector = EagerSampleSelector()
selector.sample(uncertainties, config['update_sample_size'])
X_train, y_train, X_pool, y_pool = selector.update_sets(
    X_train, y_train, X_pool, y_pool, uncertainties, config['update_sample_size']
)

array([131, 359, 134, 807, 376, 838, 635, 401, 674, 654])

In [51]:
nn.train(
    session = sess,
    X_train = X_train,
    y_train = y_train,
    X_test = X_test,
    y_test = y_test,
    X_val = X_test,
    y_val = y_test,
)

rmses = [np.sqrt(mse(nn.predict(sess, data = X_test), y_test))]

for al_iteration in range(1, 10):
    print(f'[{al_iteration}] BEFORE:')
    print('shapes:', X_train.shape, y_train.shape)
    print('shapes:', Xt_test.shape, y_test.shape)
    print('shapes:', X_pool.shape, y_pool.shape)
    
    gpue = nn.get_nngp_UE(sess, X_train, y_train, X_pool, y_pool)
    
    X_train, y_train, X_pool, y_pool = \
        update_learning_sets(X_train,
                             y_train,
                             X_pool,
                             y_pool,
                             gpue,
                             sample_size = 100)
    
    print(f'[{al_iteration}] AFTER:')
    print('shapes:', X_train.shape, y_train.shape)
    print('shapes:', X_test.shape, y_test.shape)
    print('shapes:', X_pool.shape, y_pool.shape)
    
    nn.train(
        session = sess,
        X_train = X_train,
        y_train = y_train,
        X_test = X_test,
        y_test = y_test,
        X_val = X_test,
        y_val = y_test,
    )
    rmses.append(np.sqrt(mse(nn.predict(sess, data = X_test),
            y_test)))

[100] RMSE train:197.140 test:200.312 val:200.312 patience:3
[200] RMSE train:189.607 test:192.727 val:192.727 patience:3
[300] RMSE train:170.706 test:173.730 val:173.730 patience:3
[400] RMSE train:134.434 test:137.446 val:137.446 patience:3
[500] RMSE train:90.492 test:94.209 val:94.209 patience:3
[600] RMSE train:69.340 test:74.389 val:74.389 patience:3
[700] RMSE train:67.135 test:72.645 val:72.645 patience:3
[800] RMSE train:67.009 test:72.614 val:72.614 patience:3
[900] RMSE train:66.926 test:72.577 val:72.577 patience:3
[1000] RMSE train:66.839 test:72.534 val:72.534 patience:3
[1100] RMSE train:66.749 test:72.490 val:72.490 patience:3
[1200] RMSE train:66.655 test:72.445 val:72.445 patience:3
[1300] RMSE train:66.558 test:72.399 val:72.399 patience:3
[1400] RMSE train:66.458 test:72.354 val:72.354 patience:3
[1500] RMSE train:66.356 test:72.308 val:72.308 patience:3
[1600] RMSE train:66.252 test:72.262 val:72.262 patience:3
[1700] RMSE train:66.147 test:72.218 val:72.218 patie

NameError: name 'Xt_test' is not defined

In [25]:
rmses

[67.01624050792357,
 25.617914060579913,
 20.79083550341267,
 20.592644604381892,
 19.891202936027778,
 19.169725896655404,
 19.088115981367878,
 18.436352037896626,
 15.836635818902577,
 15.385584932315856]

In [26]:
nn.predict(sess,
           data = X_test[:3])

array([[202.12682],
       [175.21193],
       [360.3318 ]], dtype=float32)

In [27]:
y_test[:3]

array([[183.97308059],
       [186.273626  ],
       [310.47204144]])

In [28]:
gpue = nn.get_nngp_UE(sess, X_train, y_train, X_pool, y_pool)
mcd_ue = nn.get_mcd_UE(sess, X_pool)
gpue, mcd_ue

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([280.89924695, 240.32911521, 212.78439218, 336.67438452,
        371.95826913, 221.99342952, 239.36145661, 175.08057346,
        250.84419167, 334.66508059, 183.36869235, 205.28629383,
        202.90857136, 292.08283795, 174.96503579, 360.71784888,
        194.59085206, 110.29008744, 270.02842567, 272.92573225,
        283.68638707, 228.97726692, 169.21232216, 241.86923846,
        268.76122167, 182.70312032, 208.02437352, 217.26213109,
        285.40837133, 244.2959037 , 254.17450913, 227.31588614,
        324.9356297 , 315.27853122, 24

In [29]:
print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

shapes: (1100, 10) (1100, 1)
shapes: (200, 10) (200, 1)
shapes: (100, 10) (100, 1)


In [30]:
X_train, y_train, X_pool, y_pool = \
    update_learning_sets(X_train,
                         y_train,
                         X_pool,
                         y_pool,
                         gpue,
                         sample_size = 10)

In [31]:
print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

shapes: (1110, 10) (1110, 1)
shapes: (200, 10) (200, 1)
shapes: (90, 10) (90, 1)
